<center><img src="./logo.png"></center>

<center><h2>Estructures Aeroespacials</h2></center>
<center><h1 style="color: red">The Staircase Project</h1></center>
<center><h4>March 2025</h4></center>
<center><h4>Joel Garcia Martin</h4></center>

___Note___: This assignment was intended to be done using MATLAB, but after asking the professor he allowed me to do it in Python. Must be noted that the syntaxis and functions are very different in some cases and because of that I have had to rewrite some of the initial conditions and functions given with this assignment. 

Doing this in python was mostly a challenge to myself, as I would need to understand the equations to write this as most of the instructions we were given were in MATLAB.

Most of the code is commented with explanations, as there is not much to say about the functions and provided code

# Introduction
In the Netflix adaptation of The Three-Body Problem book series, the protagonists attempt to send a preserved human brain into space to intercept an incoming alien fleet. Their goal is to accelerate the probe containing the brain to approximately 1% lightspeed (c=3x10^8 m/s) in order to reach the fleet within a reasonable timeframe. The idea behind the Staircase Project is to propel the probe using a series of nuclear explosions to transfer the momentum of the plasma ejected to a solar sail. In this assignment, we will study the feasibility of this project from a structural point of view.

# Estimation of the thrust force
To make an estimation of the thrust force we will make the following assumptions:
- Each detonation ejects mp=4.468 x 10^6 kg of plasma (more or less equivalent to a 10-megaton explosion).
- The plasma is released within tp=0.1 seconds in all directions as a sinusoidal pulse (half-period) advancing at a speed of Vp = 125000 km/s (see Figure 1).
- The plasma pulse reaches the probe at a distance d = 200 km.
- The total mass of the probe and solar sail is  kg, and the pusher plate surface is S = 900 m^2.

<center><img src="./image.png" style="width: 150px; background-color: white"></center>
<center>Figure 1: Plasma mass ejection rate</center>

Based on these hypothesis, the mass ejection rate of each nuclear explosion is given by:

$`\dot m_p(t) = \frac{\pi}{2}\frac{m_p}{t_p}sin(\frac{\pi t}{t_p})`$

Considering $`S<<d^2`$ and $`V<<V_p`$, from the conservation of linear momentum:

$`M\frac{dV}{dt} \approx \dot m_p(t)\frac{S}{4 \pi d^2}V_p`$

Integrating both sides over a period  we can obtain the impulse obtained from each nuclear detonation:

$`\Delta V \approx \frac{m_p}{M}\frac{S}{4 \pi d^2}V_p = 10km/s`$

Acording to these calculations, with 300 concatenated explosions (the number given in the fiction), the probe would reach a velocity close to V=3000 km/s. However, if the detonations are produced at enough distance from one another, the thrust transfered to the structure in each one will be the same. Assuming the force is distributed uniformly over the pusher plate surface, the pressure is given by: 

$` p_T(t) = \frac{\dot m_p(t)}{4 \pi d^2}V_p`$

# Solar Sail Design
Figure 2 shows a schematic representation of the solar sail. The total mass of the solar sail is M = 100 kg, which includes the mass of the structure (bar elements), $`M_s`$. The remaining mass, $`M_R = M - M_s`$ , which accounts for the payload, the communications antenna, etc., is distributed: 20% to node 1, 40% to nodes 2, 3, 4 and 5 (10% each), and 40% to node 6.  Reference material properties for each structural component are given in Table 2.


<img src="./table.png" style="width: 300px; background-color: white">

<center><img src="./download.png" style="width: 800px; background-color: white"></center>

Figure 1. Schematic representation of the solar sail structure. Data: L = 30 m, H = 5 m, l = 1 m, h = 1 m, a = 50 mm, b = 25 mm, w = 3 mm, $`D_a =`$ 75 mm, $`d_a =`$ 50 mm, $`D_p =`$ 15 mm, $`d_p =`$ 9 mm, $`D_c =`$ 5 mm.

Considering that, in each impulse, the thrust force is distributed uniformly over the the pusher plate surface (acting as a time-dependent pressure $`p_T(t)`$ defined previously). The following is asked:
- Plot of the deformed structure and stress-state of each element in the most critical scenario. [Optionally, get the results in a sequence of time instants of interest]
- Discuss whether the proposed design is capable of fulfilling the mission according to the hypothesis considered. Are material properties considered realistic? What changes on the solar sail structural design would you suggest to improve the chances of success of the mission? [Example: changing material or geometrical features, adding prestress to elements at risk of buckling, etc.]

Justify your arguments with quantitative metrics.

# The Code

We start by importing some libraries for graphing (matplotlib) and math functions (numpy).

In [39]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3DCollection

## Global Definitions

Using provided arrays in the assignment, we will define all necessary values and arrays for this simulation. After the definitions, we will create our own solve function, to compute the solution of each problem and a helper function to plot the deformation and color using the stress state.

As the plot function was given in MATLAB, I have had to rebuilt it in Python

### Global Variables

In [40]:
# Define data values
h = 1
l = 1
H = 5
L = 30

# Nodal coordinates
xnod = np.array([
    [0,  0,  0],
    [0,  h, -l/2],
    [l/2, h,  0],
    [0,  h,  l/2],
    [-l/2, h,  0],
    [0, -H,  0],
    [-L/2, h, -L/2], 
    [L/2, h, -L/2],
    [L/2, h,  L/2],
    [-L/2, h,  L/2],
])

# Nodal connectivities
Tnod = np.array([
    [0, 1],
    [0, 2],
    [0, 3],
    [0, 4],
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 1],
    [0, 5],
    [4, 6],
    [1, 6],
    [1, 7],
    [2, 7],
    [2, 8],
    [3, 8],
    [3, 9],
    [4, 9],
    [6, 7],
    [7, 8],
    [8, 9],
    [9, 6],
    [5, 6],
    [5, 7],
    [5, 8],
    [5, 9],
    [0, 6],
    [0, 7],
    [0, 8],
    [0, 9],
])

# probe structure
b_mat1 = 25e-3 # m
a_mat1 = 50e-3 # m
w_mat1 = 3e-3 # m
# Antenna mast:
Da_mat2 = 75e-3 # m
da_mat2 = 50e-3 # m
# Pusher plate structure:
Dp_mat3 = 15e-3 # m
dp_ma3 = 9e-3 # m
# Support Cables:
Dc_mat4 = 5e-3 # m

mat = np.array([
    # E, A, density, sigma_0
    [70e9, ((a_mat1 + w_mat1) * w_mat1 + b_mat1 * w_mat1 * 2), 2275, 0],
    [85e9, np.pi * ((Da_mat2/2) ** 2) - np.pi * ((da_mat2/2) ** 2), 2400, 0],
    [100e9, np.pi * (Dp_mat3/2) ** 2 - np.pi * (dp_ma3/2) ** 2, 1100, 0],
    [1000e9, np.pi * (Dc_mat4/2) ** 2, 1000, 0],
])

# Material connectivities
Tmat = [
    0,  # Probe (index 0)
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,  # Antenna mast (index 1)
    2,  # Pusher plate structure (index 2)
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    3,  # Support cables (index 3)
    3,
    3,
    3,
    3,
    3,
    3,
    3
]

nnod = xnod.shape[0] # Number of nodes
ni = 3
nel = Tnod.shape[0] # Number of Elements
nne = Tnod.shape[1]
ndof = nnod * ni
nde = nne * ni
# Equivalent of MATLAB's repelem(Tnod, [1, ni])
Tnod_repeated = np.repeat(Tnod, ni, axis=1)

# Equivalent of MATLAB's repmat(1:ni, size(Tnod))
dof_matrix = np.tile(np.arange(1, ni + 1), (Tnod_repeated.shape[0], 2))

# Compute Tdof
Tdof = ni * (Tnod_repeated) + dof_matrix - 1

#### Solver Function (same as 2D case, but updated for 3D):

In [41]:
# Solver
# Here we are going to create a function that will solve the problem for us. This is done following provided guidelines.
def solver(xnod, Tnod, mat, Tmat, fixnod, fdata):

    Kels = np.zeros((nde, nde, nel)) # List of Ks for each element
    Fels = np.zeros((nde, nel)) # List of Forces for each element

    K = np.zeros((ndof, ndof))   # Global K matrix
    f = np.zeros((ndof, 1))     # Global Forces matrix
    u = np.zeros((ndof, 1))     # Global displacements matrix
    strain = np.zeros((nel, 1))     # Global strain matrix
    vr = []                     # Restricted degrees of freedom

    sigmas = np.zeros((nel, 1))   # Stress Matrix (Result of the solver)
    for e in range(nel):           # For every element
        x1, y1, z1 = xnod[Tnod[e][0]]  # Get coordinates
        x2, y2, z2 = xnod[Tnod[e][1]]  # Get coordinates
        E, A, d, sigma_0 = mat[Tmat[e]] # Get Material Info
        l = np.sqrt((x2- x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2) # Compute its length
        cx = (x2 - x1)/l  # cx for rotation matrix
        cy = (y2 - y1)/l  # cy for rotation matrix
        cz = (z2 - z1)/l  # cz for rotation matrix
        R = np.array([
            [cx, cy, cz, 0 , 0 , 0],
            [0 , 0 , 0 , cx, cy, cz],
            ]) # Rotation Matrix
        
        Fels[:, e] = - sigma_0 * A * np.matmul(R.T, np.array([-1, 1])) # Computing initial forces from stresses
        Kels[:, :, e] = ((E * A)/l) * np.matmul(np.matmul(R.T, np.array([
            [1, -1],
            [-1, 1],
        ])), R) # Computing K matrix of element

    for e in range(nel): # For every Element
        I = Tdof[e, :] # Get degrees of freedom
        f[I] += np.array(Fels)[:, e].reshape(-1, 1) # Add Forces to global matrix using degrees of freedom
        K[np.ix_(I, I)] += Kels[:, :, e] 

    for node, dof, force in fdata: # for every force in fdata
        gdof = int(ni * (node) + dof) # Get degree of freedom of node
        f[gdof] += force  # Add force to global force vector

    for node, node_dof, displ in fixnod: # for every fixed node in fixnod
        gdof = int(ni * (node) + node_dof) # Get degree of freedom of node
        u[gdof] = displ    # Set displacement of node
        if gdof not in vr:
            vr.append(gdof) # Add to restricted degrees of freedom

    vf = np.setdiff1d(np.arange(ndof), vr) # Free degrees of freedom

    # Dividing Matrix into free and restricted:
    K_ff = K[np.ix_(vf, vf)]
    K_fr = K[np.ix_(vf, vr)]
    f_f = f[vf]

    # Computing The displacement Matrix
    u[vf] = np.linalg.inv(K_ff) @ (f_f - K_fr @ u[vr])

    # Computing Reaction forces
    r = K[vr, :] @ u - f[vr]
    
    # Next, we will compute stress:
    for e in range(nel):
        x1, y1, z1 = xnod[Tnod[e][0]]  # Get coordinates
        x2, y2, z2 = xnod[Tnod[e][1]]  # Get coordinates
        E, A, d, sigma_0 = mat[Tmat[e]] # Get Material Info
        l = np.sqrt((x2- x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2) # Compute its length
        cx = (x2 - x1)/l  # cx for rotation matrix
        cy = (y2 - y1)/l  # cy for rotation matrix
        cz = (z2 - z1)/l  # cz for rotation matrix
        R = np.array([
            [cx, cy, cz, 0 , 0 , 0],
            [0 , 0 , 0 , cx, cy, cz],
            ]) # Rotation Matrix
        uel = u[I]  # Displacement of element
        uel_prime = R @ uel  # Displacement of element in global coordinates
        epsilon = (uel_prime[1] - uel_prime[0])/l # Computing strain
        strain[e] = epsilon
        sigmas[e] = sigma_0 + E * epsilon # Computing stress from strain
    return u, r, sigmas, strain

In [ ]:
def plot3DBars(xnod, Tnod, disp, stress, scale, units, name, time):
    # Number of steps
    N = disp.shape[1]
    
    # Precomputations
    X, Y, Z = xnod[:, 0], xnod[:, 1], xnod[:, 2]
    Ux = scale * disp[0::3, 0]
    Uy = scale * disp[1::3, 0]
    Uz = scale * disp[2::3, 0]
    smin, smax = np.min(stress), np.max(stress)
    
    xmax = np.max([np.max(X), np.max(X + scale * disp[0::3, :])])
    xmin = np.min([np.min(X), np.min(X + scale * disp[0::3, :])])
    ymax = np.max([np.max(Y), np.max(Y + scale * disp[1::3, :])])
    ymin = np.min([np.min(Y), np.min(Y + scale * disp[1::3, :])])
    zmax = np.max([np.max(Z), np.max(Z + scale * disp[2::3, :])])
    zmin = np.min([np.min(Z), np.min(Z + scale * disp[2::3, :])])
    
    # Open plot window
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_box_aspect([1,1,1])
    ax.view_init(elev=15, azim=30)
    
    # Plot undeformed structure
    for elem in Tnod:
        ax.plot(X[elem], Y[elem], Z[elem], color="#ffffff", linewidth=0.5)
    
    # Plot deformed structure
    lines = [list(zip(X[elem] + Ux[elem], Y[elem] + Uy[elem], Z[elem] + Uz[elem])) for elem in Tnod]
    lc = Line3DCollection(lines, cmap='coolwarm', linewidth=1)
    lc.set_array(stress[:, 0])
    ax.add_collection(lc)
    
    # Set limits
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.set_zlim([zmin, zmax])
    
    # Colorbar settings
    cb = fig.colorbar(lc, ax=ax, shrink=0.5, aspect=10)
    cb.set_label(f'Stress ({units})')
    
    # Labels
    ax.set_title(f'scale = {scale} (σ_min = {smin:.3g} {units} | σ_max = {smax:.3g} {units} | time {time} s)')
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_zlabel('z (m)')
    ax.set_ylim((-6, 1))
    
    
    # Animate different timesteps (at the end, I ended up just doing one at a time and then joining
    # all the photos together with a video editing software)
    for i in range(N):
        Ux = scale * disp[0::3, i]
        Uy = scale * disp[1::3, i]
        Uz = scale * disp[2::3, i]
        
        lines = [list(zip(X[elem] + Ux[elem], Y[elem] + Uy[elem], Z[elem] + Uz[elem])) for elem in Tnod]
        lc.set_segments(lines)
        lc.set_array(stress[:, i])
        ax.set_title(f'scale = {scale} (σ_min = {np.min(stress[:, i]):.3g} {units} | σ_max = {np.max(stress[:, i]):.3g} {units} | time {time} s)')
        #plt.draw()
        #plt.pause(0.1)
    
    plt.savefig(f"{name}.png")
    # plt.show()

## Problem definition

First, we are going to assign the masses to each node, using the given densities and knowing that the total mass of the Solar Sail is 100Kg. This together with knowing that the rest of the mass is distributed according to the given percentages, we are able to compute how much mass each node will have for this simulation

In [43]:
mass_distribution = np.zeros((nnod)) # Empty List
for e in range(nel):
    node_0 = Tnod[e][0] # Node 0 of element
    node_1 = Tnod[e][1] # Node 1 of element
    x1, y1, z1 = xnod[node_0]  # Get coordinates
    x2, y2, z2 = xnod[node_1]  # Get coordinates
    E, A, d, sigma_0 = mat[Tmat[e]] # Get Material Info
    l = np.sqrt((x2- x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2) # Compute its length
    element_mass = d * l * A # Compute the mass of the element

    # Give half the mass to Node 0 and the other half to Node 1 for each element:
    mass_distribution[node_0] += (1/2) * element_mass 
    mass_distribution[node_1] += (1/2) * element_mass

# Knowing that the total mass of the sail is 100 kg,
MT = 100

# The rest of mass we have left over is:
Leftover = MT - np.sum(mass_distribution)

# So we can distribute it to each node according to the specification:
mass_distribution[0] += 0.2 * Leftover
mass_distribution[1] += 0.1 * Leftover
mass_distribution[2] += 0.1 * Leftover
mass_distribution[3] += 0.1 * Leftover
mass_distribution[4] += 0.1 * Leftover
mass_distribution[5] += 0.4 * Leftover

# From here, I assign a percentage of the total mass that each node will have.
mass_percentages = mass_distribution / MT
mass_percentages, np.sum(mass_percentages)

(array([0.22396791, 0.06117869, 0.06117869, 0.06117869, 0.06117869,
        0.26121356, 0.06752594, 0.06752594, 0.06752594, 0.06752594]),
 np.float64(1.0))

In [ ]:
# Variables given:
total_area = L ** 2
d_force = 200e3 # m
Vp = 125000e3 #m/s
Mp = 4.468e6 # kg
tp = 0.1
S = 900 # m^2
M = 100 # kg

# Formula definition:
mp = lambda t: (np.pi/2)*(Mp/tp)*np.sin((np.pi * t )/tp)
p = lambda t: (mp(t)/(4*np.pi*d_force ** 2))*Vp
linear_momentum = lambda t: (mp(t) * S * Vp)/(4 * np.pi * d_force ** 2)

# This function will give us the fdata for a given timestamp
def get_fdata(t):

    # For each node we take into account two forces: Inertial forces, 
    # that will want to keep the structure as it is and pressure forces, 
    # that want to push the structure forward. Pressure forces have a 
    # positive sign in the y direction and intertial will have negative sign.

    fdata = np.array([ 
        [0, 1, - linear_momentum(t) * mass_percentages[0]],
        [1, 1, total_area*(3/16)*p(t) - linear_momentum(t) * mass_percentages[1]],
        [2, 1, total_area*(3/16)*p(t) - linear_momentum(t) * mass_percentages[2]],
        [3, 1, total_area*(3/16)*p(t) - linear_momentum(t) * mass_percentages[3]],
        [4, 1, total_area*(3/16)*p(t) - linear_momentum(t) * mass_percentages[4]],
        [5, 1,  - linear_momentum(t) * mass_percentages[5]],
        [6, 1, total_area*(1/16)*p(t) - linear_momentum(t) * mass_percentages[6]],
        [7, 1, total_area*(1/16)*p(t) - linear_momentum(t) * mass_percentages[7]],
        [8, 1, total_area*(1/16)*p(t) - linear_momentum(t) * mass_percentages[8]],
        [9, 1, total_area*(1/16)*p(t) - linear_momentum(t) * mass_percentages[9]],
    ])
    return fdata

fixnod = np.array([
    # node, axis, displ
    [0, 0, 0],
    [0, 1, 0],
    [0, 2, 0],
    [3, 0, 0],
    [5, 2, 0],
    [5, 0, 0],
]) # Fixed Nodes

## Computing the solution to the problem

Now, we will use the solver we created previously to compute the solution to the problem, obtaining displacements, reactions and stresses.

In [ ]:
for time in np.arange(0.0, 0.1, 0.01):
    fdata = get_fdata(time) # we get the data
    disp, reac, stress, strain = solver(xnod, Tnod, mat, Tmat, fixnod, fdata) # we solve the problem
    plot3DBars(xnod, Tnod, disp, stress*1e-9, 1, "GPa", f"./imgs/{time}_IMG", time)

Where the result we get is the following:

<img src="./AO2.gif">

## Discussion:

The results of this simulation show extreme deformation, to the point that with scale 1 you can observe the entire structure deforming around 2 meters in the y direction. Stresses are also very high for this kind of material, so the conclusion on how to improve this design is to add more nodes or increase the material strength or area of those in the sail. The bar that handles most of the pressure is the one between node 1 and 6 in Figure 1, which is also where the sail broke in the movie (some of the bolts got loose).

One improvement that could increase the chances of success of this mission would be to make everything out of a higher strength material like material number 4. Sure it would increase weight, but otherwise this mission will fail.